## Chapter 2
### Multi-armed Bandits

This chapter aims to discuss the fundamental RL problem, called the 'Multi-armed Bandit', a reference to a slot machine with multiple levers. The goal here is to discuss and examine techniques for solving the problem in which a fixed number of decisions at every time interval combine to a final reward, which we aim to maximize. 

## Notation

$t$ = time step.

$A_t$ = action selected at $t$.

$R_t$ = reward at time $t$.

$q_*(a)$ = expected reward of arbitrary action $a$. Defined by: 
$$ q_*(a) = \mathbb{E}[R_t|A_t =a ] $$

$Q_t(a)$ = estimated value of action $a$ at time $t$.



## Action-value Methods

Since the value of each possible is action is not always (or ever) known at each time step $t$, there is a need to find the best technique by which to estimate the rewards from actions, and thus to decide which actions to take. The collection of such estimations are called *action-value methods*. 

This simplest action selection rule is to select the action with the highest-observed value. This is called the greedy action, and this method can be summarized as

$$ A_t = \text{argmax}_a Q_t(a).$$

As an alternative, we define the $\epsilon$-greedy method, which says that with some small probability $\epsilon$, we choose an action from all available actions uniformally at random, and otherwise we act greedily. 

### Exercise 2.1

In $\epsilon$-greedy action selection, for the case of two actions and $\epsilon=.5$, what is the probability that the greedy action is selected?

#### Solution
For this, we use law of total probablity. Let $a_g$ be the event that the greedy action is selected, and let $a_\epsilon$ be the event that the action is selected at random. Then
$$ P(a_g) = P(a_g|a_\epsilon)\cdot P(a_\epsilon) + P(a_g|a_\epsilon ')\cdot P(a_\epsilon')$$
$$ = \frac{1}{2}\cdot\frac{1}{2}+1\cdot\frac{1}{2} = \frac14$$

## Exercise 2.2

Consider a $k$-armed bandit problem with $k=4$ actions, denoted 1,2,3,4. Consider applying an $\epsilon$-greedy action selection, using sample-average action-value estimates, and initial estimates of $Q_1(a) = 0$ for all $a$. Suppose an initial sequence of actions and rewards is given by:

$A_1 = 1, R_1 = -1$

$A_2 = 2, R_2 = 1$

$A_3 = 2, R_3 = -2$

$A_4 = 2, R_4 = 2$

$A_5 = 3, R_5 = 0$

On which time steps did the $\epsilon$ case definitely occur? On which time steps could this have possible occurred? 

#### Solution
Answering the second part first, the $\epsilon$ case could have occurred at every time step. 

There is no way to determine if it occurred at time step 1, since each action-value estimate is equal at 0 at that point.

At time step 2, action 2 could have been a legal choice of the greedy algorithm, so again we cannot know which case occurred.

At time step 3, action 2 is the most rewarding action thus far. It again could have been the result of either case.

At time step 4, action 2 is now the second-least rewarding action, behind actions 3 and 4. Thus, we know for certain that the $\epsilon$ case occurred here. 

At time step 5, action 2 has returned to being the most rewarding action, with an expected payout of $\frac13$. Thus, since action 3 was chosen, we know that the $\epsilon$ case occurred.

### Exercise 2.3
In the comparison shown in Figure 2.2 (seen on page 30 in the textbook), which method will perform best in the long run in terms of cumulative reward and probability of selecting the best action? How much better will it be? Express your answer quantitatively.

#### Solution

In [1]:
import numpy as np

q-stars = np.random.normal(0,1,10)


array([-0.06743078,  0.68999754, -1.25054989, -1.62882987, -2.0087425 ,
       -1.40334354,  0.30859429,  2.3377153 , -1.14725134,  1.16663608])